<a href="https://colab.research.google.com/github/KirtiNayak11/NLP/blob/main/fine_tune_t5_for_conversational_model_using_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model

In [ ]:
df = pd.read_csv('/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv')

In [ ]:
df.head()

,Unnamed: 0,question,answer
0,0,"hi, how are you doing?",i'm fine. how about yourself?
1,1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,3,no problem. so how have you been?,i've been great. what about you?
4,4,i've been great. what about you?,i've been good. i'm in school right now.


In [ ]:
df.shape

(3725, 3)

In [ ]:
df.columns

Index(['Unnamed: 0', 'question', 'answer'], dtype='object')

In [ ]:
df.drop(columns = ['Unnamed: 0'],inplace = True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
print(df[df.duplicated()])
print(df.duplicated().sum())

                          question    answer
1749  i don't like riding the bus.  why not?
1


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.isna().sum()

question    0
answer      0
dtype: int64

In [ ]:
df.shape

(3724, 2)

Data Preprocessing

1.Lowercase all characters

2.Remove quotes

3.Remove all the special characters

4.Remove all numbers from text

5.Remove extra spaces


In [ ]:
import re

In [ ]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub("'",'',text)
    text = re.sub(r'[^a-z\s]','',text)
    text = re.sub(r'\s+',' ',text).strip()
    return text

df = df.map(preprocess_text)

In [ ]:
df['answer'] = df['answer'].apply(lambda x:'START_' + x + '_END')

In [ ]:
df.head()

,question,answer
0,hi how are you doing,START_im fine how about yourself_END
1,im fine how about yourself,START_im pretty good thanks for asking_END
2,im pretty good thanks for asking,START_no problem so how have you been_END
3,no problem so how have you been,START_ive been great what about you_END
4,ive been great what about you,START_ive been good im in school right now_END


In [ ]:
import tensorflow as tf

In [ ]:
def tokenize(text):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(text)
    total_words = (tokenizer.word_index)
    tensor = tokenizer.texts_to_sequences(text)[0]
    max_seq_len = max([len(x) for x in tensor])
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,\
                                                           padding = 'post',\
                                                           maxlen = max_seq_len,\
                                                           dtype='int32')

    return total_words,tensor,tokenizer

In [ ]:
total_input_words,input_sequence,input_tokenizer = tokenize(df['question'].tolist())
total_output_words,output_sequence,output_tokenizer = tokenize(df['answer'].tolist())

In [ ]:
input_vocab_size = len(total_input_words) + 1
output_vocab_size = len(total_output_words) + 1
max_input_length = input_sequence.shape[1]
max_output_length = output_sequence.shape[1]

In [ ]:
lstm_units = 256

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM

In [ ]:
encoder_inputs = Input(shape = (max_input_length,))
encoder_embedding_layer = Embedding(input_dim = input_vocab_size,output_dim = 100,mask_zero = True)(encoder_inputs)
encoder = LSTM(lstm_units,return_state=True)
encoder_outputs,state_h,state_c = encoder(encoder_embedding_layer)
encoder_states = [state_h,state_c]

In [ ]:
lstm_units=256

encoder_inputs = Input(shape = (input_sequence.shape[1],))
embedding_layer = Embedding(input_dim = len(total_input_words)+1,output_dim = 100,mask_zero = True)(encoder_inputs)
encoder = LSTM(lstm_units,return_state=True)
encoder_outputs,state_h,state_c = encoder(embedding_layer)
encoder_states = [state_h,state_c]

encoder_model = Model(encoder_inputs,encoder_states)

decoder_inputs = Input(shape = (max_output_length,))
decoder_embedding_layer = Embedding(input_dim = output_vocab_size,output_dim = 100,mask_zero = True)(decoder_inputs)

decoder = LSTM(lstm_units,return_sequences = True,return_state = True)
decoder_outputs,state_h,state_c = decoder(decoder_embedding_layer,initial_state=encoder_states)

decoder_dense = Dense(output_vocab_size,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Print summary
model.summary()

# ---- TRAINING ----
# Convert decoder output to one-hot encoding
decoder_target_data = tf.keras.utils.to_categorical(output_sequence, num_classes=output_vocab_size)



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 19)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 19)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 19, 100)        │        233,800 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_1 (NotEqual)    │ (None, 19)             │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 19, 100)        │        349,000 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 256), (None,   │        365,568 │ embedding_1[0][0],     │
│                           │ 256), (None, 256)]     │                │ not_equal_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 19, 256),      │        365,568 │ embedding_2[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_1[0][1],          │
│                           │ 256)]                  │                │ lstm_1[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 19, 3490)       │        896,930 │ lstm_2[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,210,866 (8.43 MB)

 Trainable params: 2,210,866 (8.43 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import os
import tensorflow as tf

checkpoint_path = "/kaggle/working/model_checkpoint.keras"  # Change to .keras
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=False,  # Save full model
                                                 verbose=1)

In [ ]:
# Train the model
model.fit([input_sequence, output_sequence], decoder_target_data,
          batch_size=64, epochs=50, validation_split=0.2,callbacks=[cp_callback]
         )

Epoch 1/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step - accuracy: 0.0138 - loss: 8.0295
Epoch 1: saving model to /kaggle/working/model_checkpoint.keras
47/47 ━━━━━━━━━━━━━━━━━━━━ 25s 450ms/step - accuracy: 0.0138 - loss: 8.0218 - val_accuracy: 0.0185 - val_loss: 6.8338
Epoch 2/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.0182 - loss: 6.5264
Epoch 2: saving model to /kaggle/working/model_checkpoint.keras
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 418ms/step - accuracy: 0.0182 - loss: 6.5247 - val_accuracy: 0.0205 - val_loss: 6.5610
Epoch 3/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step - accuracy: 0.0214 - loss: 6.2542
Epoch 3: saving model to /kaggle/working/model_checkpoint.keras
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 430ms/step - accuracy: 0.0214 - loss: 6.2535 - val_accuracy: 0.0195 - val_loss: 6.4831
Epoch 4/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.0214 - loss: 6.1228
Epoch 4: saving model to /kaggle/working/model_checkpoint.keras
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 426ms/step - ac

In [ ]:
# # Loads the weights
model.load_weights('/kaggle/working/cp.ckpt')

In [ ]:
decoder_inputs = Input(shape = (output_sequence.shape[1],))
decoder_embedding_layer = Embedding(input_dim = len(total_output_words)+1,output_dim = 100,mask_zero = True)(decoder_inputs)

decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))

decoder_input_state = [decoder_state_input_h, decoder_state_input_c]

decoder = LSTM(lstm_units,return_sequences = True,return_state = True)
decoder_outputs,state_h,state_c = decoder(decoder_embedding_layer,initial_state=decoder_input_state)

decoder_output_state = [state_h,state_c ]

decoder_dense = Dense(len(total_output_words)+1,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs]+decoder_input_state,[decoder_outputs]+decoder_output_state)

In [ ]:
def decode_seq(input_seq):

    states_value = encoder_model.predict(input_seq)
    print(type(states_value))

    target_seq = np.zeros((1,1))
    target_seq[0,0] = output_tokenizer.word_index['<start>']

    decode_senetence = ''

    stop_condition = False

    while not stop_condition:

        output_tokens,h,c = decoder_model.predict([target_seq]+states_value)

                                #(batch_size, sequence_length, vocab_size)
        sampled_token_index = np.argmax(output_tokens[0,-1,:])
        sampled_word = reverse_output_word_index(sampled_token_index)

        decoded_sentence+= ''+=sampled_word

        if (sampled_word=='<end>' or len(decoded_sentence) > max_output_length):
            stop_condition = True

        target_seq = np.zeros((1,1))
        target_seq[0,0] = sampled_token_index

        states_value = [h,c]

    return decoded_sentence

reverse_output_word_index = {index:word for word,index in output_tokenizer.word_index.items()}
max_output_length = decoder_input_data.shape[1]

input_sentence = "hello how are you"
input_seq = input_tokenizer.texts_to_sequences([input_sentence])
input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=max_input_length, padding='post')

decoded_sentence = decode_sequence(input_seq)
print("Predicted Output:", decoded_sentence)